# Interview Scheduler

## Assumptions
You don't care about the ordering of the interviews, I.E. you have no preference to say "I would like to go to the UF interview before the FSU interview" assuming both were chosen. What this means - this scheduler assumes if you are going to both, you don't care about which day you go if one is on say 11/12 and the other is on 11/13. It's going to randomly pick one. This probably works for virtual interviews, but it's not very realistic for physical interviews. 

## Running
If you're running in google colab (strongly recommended, click the run in colab button on this page), select a runtime in the top left under options, pick gpu as your runtime, then under runtime click "run all cells"

In [15]:

import numpy as np
import utils

school_array = np.genfromtxt('School_Date_Tracker.csv', delimiter=',',names=True,dtype=None, encoding=None)

school_name_list, date_list, prio_list = utils.parse_school_array(school_array)
school_choice_dict = {name:0 for name in school_name_list}
unique_date_list = np.sort(utils.get_unique_date_list(date_list))

match_dict = {}
for date in unique_date_list:
    #lul this is the one halfway okay piece of code in this entire notebook :) It gets its own cell
    match_dict[date] = [row[0] for row in school_array if date in row]


# Run the following cell if you'd like to see an output of all of the schools available for each date. This may be a large output. 

In [ ]:
for key in match_dict:
    print(f'Date: {key} Schools: {match_dict[key]}')

## Here comes the limit of my technical competence. Scheduling problem was doable if I wanted to look into cplex modules for python, but uh. We didn't do that. Lot of projects going on and busted this out in a few hours, don't hate me or judge me if this gets shared lul. The following cell might take a while to run. I'd recommend if you're on google colab, you go to the edit settings, and under the runtime option, you make sure "Gpu" is selected.

In [18]:
import random as rd
GIANT_NUMBER = 10000000000 #Change this value to determine how long you want this to run for
school_value = utils.get_schedule_value(prio_list, school_choice_dict)
print(f'Starting School Value: {school_value}')
prio_list = np.ones(len(prio_list))
#the idea here - is I've got a list of schools, with 0 next to them. I can map that, to the list of date dictionaries that have schools
chosen_list = []
comparison_dict = {}
optimal_dict = {}
for i in range(GIANT_NUMBER): #lul don't judge me on how hacky this is
    for key in list(match_dict.keys()):
        reference_row = [value for value in match_dict[key] if value not in chosen_list]
        if len(reference_row) != 0:
            random_choice = rd.randint(0,(len(reference_row)-1))
            choice = reference_row[random_choice]
            comparison_dict[key] = choice
            chosen_list.append(choice)

    for school in list(school_choice_dict.keys()):
        if school in chosen_list:
            school_choice_dict[school] = 1
    
    iter_value = utils.get_schedule_value(prio_list, school_choice_dict)
    if iter_value > school_value: 
        print(f'School Value for iteration: {i} improved to {iter_value}')
        school_value = iter_value
        optimal_dict = comparison_dict
        print(optimal_dict)

Starting School Value: 26.0


KeyboardInterrupt: 